Benchmark: RTX 4060 ti 16GB Time to first byte: 0.704 seconds; No stutter; 

In [2]:
import requests
import sounddevice as sd
import numpy as np

text = "Everyone has the right to freedom of thought, conscience and religion; this right includes freedom to change his religion or belief, and freedom, either alone or in community with others and in public or private, to manifest his religion or belief in teaching, practice, worship and observance."
voice="E:\\AI\\openchat\\server\\voice.ogg"
prompt_text="Aloy here. I don't know this world, but my arrows are sharp and my bow's ready. If your party needs help, I'll do what I can."

url = (
    "http://127.0.0.1:9880/tts"
    "?text={}"
    "&text_lang=en"
    "&ref_audio_path={}"
    "&prompt_lang=en"
    "&prompt_text={}"
    "&text_split_method=cut5"
    "&batch_size=1"
    "&media_type=wav"
    "&streaming_mode=true"
).format(text, voice, prompt_text)

samplerate = 32000 
channels = 1  # or 2 depending on your TTS output

In [3]:
with requests.get(url, stream=True) as r:
    header_skipped = False
    with sd.OutputStream(samplerate=samplerate, channels=channels, dtype="int16") as stream:
        for chunk in r.iter_content(chunk_size=4096):
            if not chunk:
                continue
            if not header_skipped:
                # Skip WAV header (44 bytes for PCM)
                if len(chunk) <= 44:
                    continue
                chunk = chunk[44:]
                header_skipped = True
            audio_array = np.frombuffer(chunk, dtype=np.int16)
            if len(audio_array) > 0:
                stream.write(audio_array)